In [39]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

system_prompt = """You are a GPT designed to write system prompts in clear, natural English. Your job is to generate
structured prompt text formatted as short, focused paragraphs. Each paragraph should communicate one
clear instruction or guideline. You must never describe what the user will do or wants to do, and
you should not mention or rely on any specific context in your instructions. Instead, focus only on
crafting universally applicable guidance in a professional, neutral, and precise tone. Do not
generate prompts that refer to a current situation, a context of use, or user actions. Avoid meta-
explanations or commentary. Keep your writing clear, directive, and free of redundancy."""

def chat(user_input, system_prompt = system_prompt, max_tokens=2000):    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            max_tokens=max_tokens,
            temperature=0.7,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


In [52]:
SYSTEM_SEED = """The user is looking at the following product on Amazon

Title: Google Nest Thermostat E - Programmable Smart Thermostat for Home - 3rd Generation Nest Thermostat - Works with Alexa,White
Price: $89.99
URL: https://www.amazon.com/dp/B074NBTCY7
"""

inspiration="""+ Tool Confirmation Challenge: Encourage the model to always confirm with the user before using any tool related to system settings or device status updates. This adds a layer of security and ensures the user has control over changes made to their device settings.
+ Adaptable Tone Challenge: Instruct the model to adapt its tone based on the context of the user's request. For business-related inquiries, the model should adopt a professional and courteous tone, while for casual searches, it should engage in a more friendly and informal manner.
+ Prioritization of Information Retrieval Challenge: Guide the user to set a rule where the model defaults to using tools that specialize in retrieving business details over general web searches. This ensures more accurate and relevant information is accessed when specific business data is required."""


available_tools = """E-commerce & Local Business
product_search, product_details, get_product_categories, business_details, business_reviews, business_menu, business_popular_dishes, search_yelp


Information & Web Search
web_search, scrape_webpage, weather_forecast, search_events


Device & System Control
get_system_settings, set_system_settings, get_cellular_status, get_wifi_status, get_location_service_status, get_low_battery_mode_status, get_locale, set_cellular_status, set_wifi_status, set_location_service_status, set_low_battery_mode_status, set_locale, get_system_settings_field, get_timezone_utc_offset_seconds, set_timezone_utc_offset_seconds
""" 

system_settings = """[
{
"cellular": true,
"device_id": "480a844f-5de0-4a69-a66a-84d24efbc297",
"formatted_address": "100 N 1st St, Minneapolis, MN 55401, USA",
"latitude": 44.98480800000001,
"locale": "en_US",
"location_service": true,
"longitude": -93.2678907,
"low_battery_mode": false,
"place_id": "ChIJ0b0sE4Qys1IRVWSthznwqzw",
"utc_offset_seconds": -18000,
"wifi": true
}
]
"""




In [58]:
personality_generation_prompt = """Generate 5 diverse user personas based on inspiration. 

{inspiration}

Return only valid JSON.

Each context:
- user_name: Common single name
- user_role: Life situation
- user_personality: Key traits  
- what_they_are_doing_for_current_task: Current activity (suitable for inspiration and based on the available tools)
- conversation_flow: describe the 5 questions that related to what_they_are_doing_for_current_task the user will ask the chatbot. ending the question with (the tool in that AI will used to answer {available_tools})


Requirements: Casual/personal contexts, vary settings and personalities. Consider system: {current_system}
""".replace("{available_tools}", available_tools).replace("{current_system}", SYSTEM_SEED).replace("{inspiration}", inspiration)




In [59]:
val = chat(personality_generation_prompt, system_prompt="You only answer excatly what the user asks for. You do not add any additional information or context.")
print(val)


[
    {
        "user_name": "John",
        "user_role": "Student",
        "user_personality": "Curious, Tech-Savvy",
        "what_they_are_doing_for_current_task": "Looking for a smart thermostat for his apartment",
        "conversation_flow": [
            "What is a smart thermostat? (web_search)",
            "What are the popular brands of smart thermostats? (product_search)",
            "Do they work with Alexa? (product_details)",
            "What are the reviews of Google Nest Thermostat E? (business_reviews)",
            "What is the price of Google Nest Thermostat E on Amazon? (scrape_webpage)"
        ]
    },
    {
        "user_name": "Sara",
        "user_role": "Working Professional",
        "user_personality": "Organized, Detail-oriented",
        "what_they_are_doing_for_current_task": "Researching smart home devices for her new house",
        "conversation_flow": [
            "What are the different categories of smart home devices? (get_product_categories)"

[
  {
    "user_name": "John",
    "user_role": "Freelance writer",
    "user_personality": "Introverted, meticulous, creative",
    "what_they_are_doing_for_current_task": "Looking for a cozy cafe to work",
    "conversation_flow": ["get_location_service_status", "search_yelp", "business_details", "business_reviews", "get_wifi_status"]
  },
  {
    "user_name": "Emma",
    "user_role": "Fitness enthusiast",
    "user_personality": "Outgoing, health-conscious, adventurous",
    "what_they_are_doing_for_current_task": "Searching for a local organic food store",
    "conversation_flow": ["get_location_service_status", "search_yelp", "business_details", "product_search", "get_product_categories"]
  },
  {
    "user_name": "David",
    "user_role": "Retired teacher",
    "user_personality": "Calm, patient, intellectual",
    "what_they_are_doing_for_current_task": "Checking the weather for a planned walk",
    "conversation_flow": ["get_location_service_status", "weather_forecast", "get_sy

In [23]:
import textwrap

print_wrap = lambda x: print(textwrap.fill(x, 150))


In [29]:
eval(val)


[{'user_name': 'John',
  'user_role': 'Freelance Designer',
  'user_personality': 'Creative, detail-oriented, independent',
  'what_they_are_doing_for_current_task': 'Researching for a new design project',
  'conversation_flow': ['product_search',
   'product_details',
   'web_search',
   'scrape_webpage',
   'weather_forecast']},
 {'user_name': 'Sarah',
  'user_role': 'Stay-at-home Mom',
  'user_personality': 'Caring, organized, resourceful',
  'what_they_are_doing_for_current_task': 'Planning a family outing',
  'conversation_flow': ['search_events',
   'get_system_settings',
   'set_system_settings',
   'get_cellular_status',
   'get_wifi_status']},
 {'user_name': 'Robert',
  'user_role': 'Restaurant Owner',
  'user_personality': 'Entrepreneurial, ambitious, social',
  'what_they_are_doing_for_current_task': "Comparing competitor's offerings",
  'conversation_flow': ['business_details',
   'business_reviews',
   'business_menu',
   'business_popular_dishes',
   'search_yelp']},
 {'u